In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from scipy.special import expit, logit
from sklearn.metrics import roc_auc_score

print(os.listdir("input"))

['sub_stacked.csv', 'Krishna_s_CatBoost_1_1_CB_1_1.csv', 'bagging_lgbm_submit_day7_8_with_mapping_3.csv']


In [9]:
almost_zero = 1e-10
almost_one = 1 - almost_zero
LOGIT_WEIGHT = .8

In [10]:
models = {
    'file1': 'input/bagging_lgbm_submit_day7_8_with_mapping_3.csv',
    'file2': 'input/sub_stacked.csv',
    'file3': 'input/Krishna_s_CatBoost_1_1_CB_1_1.csv'
}

In [11]:
# Public leaderbaord scores, for comparison
weights = {
    'file1': 0.7,
    'file2': 0.2,
    'file3': 0.1
}

In [12]:
print (sum(weights.values()))

0.9999999999999999


In [13]:
subs = {m:pd.read_csv(models[m]) for m in models}
first_model = list(models.keys())[0]
n = subs[first_model].shape[0]

In [14]:
ranks = {s:subs[s]['is_attributed'].rank()/n for s in subs}
logits = {s:subs[s]['is_attributed'].clip(almost_zero,almost_one).apply(logit) for s in subs}

In [15]:
logit_avg = 0
rank_avg = 0
for m in models:
    s = logits[m].std()
    print(m, s)
    logit_avg = logit_avg + weights[m]*logits[m] / s
    rank_avg = rank_avg + weights[m]*ranks[m]

logit_rank_avg = logit_avg.rank()/n
final_avg = LOGIT_WEIGHT*logit_rank_avg + (1-LOGIT_WEIGHT)*rank_avg

file1 1.8079053293576095
file2 1.065925080982493
file3 1.8096665582256892


In [16]:
final_sub = pd.DataFrame()
final_sub['click_id'] = subs[first_model]['click_id']
final_sub['is_attributed'] = final_avg

In [17]:
print( final_sub.head() )

   click_id  is_attributed
0         0       0.875685
1         1       0.538684
2         2       0.148901
3         3       0.579784
4         4       0.507183


In [18]:
final_sub.to_csv("output/stacking_2.csv", index=False, float_format='%.8f')